# This code is part of the paper:
"Fatigue damage reconstruction in glass/epoxy composites via thermal analysis and machine learning: A theoretical study”, by Rodrigo Q. Albuquerque, Ali Sarhadi, Martin Demleitner, Holger Ruckdäschel and Martin A. Eder. 

# Instructions

Execute all cells below to import the dataset and perform predictions on the test set using the best preßtrained model used in the paper.

#### Code written by R.Q.Albuquerque@8December2023

In [2]:
######## LIBRARIES #########
import numpy as np
import joblib
import matplotlib.pyplot as plt

#### 1) Import the training and test sets

In [7]:
# Training and test sets already have every 5th pixel of the original images as a long 1D array
X_train = np.load('X_train.npy') # 85% of the dataset
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

#### 2) Import the best pre-trained models and scalers, then scale X_test

In [8]:
# import the GPR models
model_s11 = joblib.load('gpr_s11')
model_s33 = joblib.load('gpr_s33')
model_sumeig = joblib.load('gpr_sumeig')
model_tmax = joblib.load('gpr_tmax')

# import the X and y scalers
xscaler = joblib.load('xscaler_xtrain')
yscaler_s11 = joblib.load('yscaler_s11')
yscaler_s33 = joblib.load('yscaler_s33')
yscaler_sumeig = joblib.load('yscaler_sumeig')
yscaler_tmax = joblib.load('yscaler_tmax')

# pre-preprocess X_test
X_test_s = xscaler.transform(X_test)

#### 3) Perform predictions on the test set

In [9]:
# predict s11
s11_pred_s = model_s11.predict(X_test_s)
s11_pred = yscaler_s11.inverse_transform(s11_pred_s.reshape(-1,1))
s11_true = y_test[:,0]

# predict s33
s33_pred_s = model_s33.predict(X_test_s)
s33_pred = yscaler_s33.inverse_transform(s33_pred_s.reshape(-1,1))
s33_true = y_test[:,2]

# predict SumEig
sumeig_pred_s = model_sumeig.predict(X_test_s)
sumeig_pred = yscaler_sumeig.inverse_transform(sumeig_pred_s.reshape(-1,1))
sumeig_true = y_test[:,8]

# predict Tmax
tmax_pred_s = model_tmax.predict(X_test_s)
tmax_pred = yscaler_tmax.inverse_transform(tmax_pred_s.reshape(-1,1))
tmax_true = y_test[:,9]

#### 3) Plot the predictions

In [11]:
names = ['Tmax', 's11', 's33', 'SumEig']
true_targets = [tmax_true, s11_true, s33_true, sumeig_true]
pred_targets = [tmax_pred, s11_pred, s33_pred, sumeig_pred]

plt.figure(figsize=(8,8))
for i, (name, pred, true) in enumerate(zip(names, true_targets, pred_targets)):
    plt.subplot(2,2,i+1)
    plt.plot(pred, true, 'bo')
    low,high = true.min(), true.max()
    plt.plot([low,high],[low,high],'-r')
    plt.xlabel('Predicted target')
    plt.ylabel('True target')
    plt.title(f'Property = {name}')
plt.tight_layout()